In [7]:
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [14]:
df_rpkms = pd.read_csv('/Users/cmdb/Downloads/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz', compression = 'gzip', skiprows = 2, sep = '\t')
print(df_rpkms)

                    Name Description  GTEX-N7MS-0007-SM-2D7W1  \
0      ENSG00000223972.4     DDX11L1                 0.000000   
1      ENSG00000227232.3      WASH7P                 2.917592   
2      ENSG00000243485.1  MIR1302-11                 0.000000   
3      ENSG00000237613.2     FAM138A                 0.000000   
4      ENSG00000240361.1     OR4G11P                 0.000000   
...                  ...         ...                      ...   
52571  ENSG00000198786.2      MT-ND5               627.419189   
52572  ENSG00000198695.2      MT-ND6               553.020569   
52573  ENSG00000210194.1   J01415.21                 0.434417   
52574  ENSG00000198727.2      MT-CYB              2022.417969   
52575  ENSG00000210195.2   J01415.22                 0.000000   

       GTEX-N7MS-0011-R10A-SM-2HMJK  GTEX-N7MS-0011-R11A-SM-2HMJS  \
0                          0.000000                      0.000000   
1                          1.958602                      5.841671   
2           

In [15]:
df_rpkms.median(axis=1)

0           0.000000
1           5.917334
2           0.000000
3           0.000000
4           0.000000
            ...     
52571    2376.176025
52572    2423.041016
52573       0.342601
52574    8809.516602
52575       0.000000
Length: 52576, dtype: float64

In [19]:
roi = df_rpkms.median(axis = 1) > 0 
print(roi)

0        False
1         True
2        False
3        False
4        False
         ...  
52571     True
52572     True
52573     True
52574     True
52575    False
Length: 52576, dtype: bool


In [20]:
df_rpkms = df_rpkms.loc[roi,:]

In [21]:
df_rpkms

,Name,Description,GTEX-N7MS-0007-SM-2D7W1,GTEX-N7MS-0011-R10A-SM-2HMJK,GTEX-N7MS-0011-R11A-SM-2HMJS,GTEX-N7MS-0011-R1a-SM-2HMJG,GTEX-N7MS-0011-R2a-SM-2HML6,GTEX-N7MS-0011-R3a-SM-33HC6,GTEX-N7MS-0011-R4a-SM-2HMKW,GTEX-N7MS-0011-R5a-SM-2HMK8,...,GTEX-X4LF-0526-SM-3NMB6,GTEX-X4LF-1726-SM-3NMBZ,GTEX-X4XX-0005-SM-3NMCS,GTEX-X4XX-0011-R1B-SM-3P622,GTEX-X4XX-0011-R2A-SM-3P623,GTEX-X4XX-0126-SM-3NMC2,GTEX-X4XX-0626-SM-3NMC1,GTEX-X4XX-1126-SM-3NMBY,GTEX-X4XX-2926-SM-3NMB1,GTEX-X4XX-3026-SM-3NMB2
1,ENSG00000227232.3,WASH7P,2.917592,1.958602,5.841671,1.728239,2.315600,3.742634,2.269886,2.442356,...,9.417033,7.007399,6.276984,3.710413,5.769073,12.540883,4.696292,4.555761,9.459983,5.700174
6,ENSG00000238009.2,RP11-34P13.7,0.177576,0.204927,0.289186,0.248324,0.268730,0.168188,0.408321,0.219865,...,0.229263,0.218716,0.658286,0.314833,0.514728,0.389351,0.255082,0.248918,0.487006,0.290606
7,ENSG00000239945.1,RP11-34P13.8,0.272705,0.202653,0.510465,0.280842,0.316025,0.162210,0.715072,0.254829,...,0.270831,0.134354,0.687998,0.479412,0.533255,0.357458,0.248718,0.273047,0.591012,0.261905
8,ENSG00000237683.3,RP11-34P13.11,88.187675,1.009232,1.577608,0.575865,0.981906,0.532865,1.030279,0.767670,...,10.860014,2.222919,128.481339,2.991933,1.625518,10.951993,0.531858,4.747769,3.747678,2.664726
9,ENSG00000239906.1,RP11-34P13.14,21.158670,0.097359,0.208453,0.000000,0.227738,0.152861,0.000000,0.130077,...,1.013800,0.457205,11.639369,0.208268,0.170792,0.928909,0.126958,0.371670,0.526572,0.171122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52566,ENSG00000198840.2,MT-ND3,546.563416,9898.380859,4572.871582,10415.388672,7144.897461,6341.940430,8621.208984,10017.948242,...,3578.243652,5226.624023,870.697571,13152.982422,3865.071777,4782.514160,3418.035400,16177.149414,4064.408936,6445.033691
52571,ENSG00000198786.2,MT-ND5,627.419189,3967.486572,1014.433228,4683.750000,2813.970947,2435.256104,3997.514893,5427.200684,...,3997.743164,1589.340210,277.970520,7192.353516,1723.450684,2418.593018,3559.731689,2795.689209,1894.077515,3302.491699
52572,ENSG00000198695.2,MT-ND6,553.020569,2737.471680,787.444824,3694.725342,1631.243652,2049.827148,2676.096191,3723.490234,...,6915.279297,1867.175537,400.203766,8612.575195,1159.216675,4115.834473,5160.529297,2533.044434,1654.201904,3118.078125
52573,ENSG00000210194.1,J01415.21,0.434417,0.000000,0.243951,0.847667,0.000000,0.357784,1.051482,0.811884,...,2.372880,0.214025,0.000000,0.974938,0.199876,0.621196,1.584828,0.217481,0.410828,0.000000


In [30]:
df_metadata = pd.read_csv('/Users/cmdb/qbb2020/data/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt', sep = '\t')

In [31]:
sample_names = df_rpkms.columns.values[2:]
roi = df_metadata['SAMPID'].isin(sample_names)
df_metadata = df_metadata.loc[roi]
pca_input = df_rpkms.iloc[:,2:].to_numpy()
pca_input = pca_input.T

In [32]:
pca_input.mean(axis = 0)

array([6.68460930e+00, 3.49523801e-01, 4.27110463e-01, ...,
       3.74444996e+03, 7.61139368e-01, 1.18459041e+04])

In [37]:
pca_output = pca.fit_transform(pca_input_standardized)
pca_output_df = pd.DataFrame(data = pca_output, 
                             columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
fig, ax = plt.subplots()
ax.scatter(x = pca_output_df['PC1'], y = pca_output_df['PC2'])

NameError: name 'pca_input_standardized' is not defined